# What is about ?

Blend with scipy optimization added as option.

V10 C [0.5, 1, 2, 4 ], n_blends = 1

V9 - n_blends = 3

V8 - submitted got improvement

V6 test run with change blend to simple 

v4 - added check in blend for worse than constant 0.016306	0.016173 0.016062	0.016080

V3 60 minutes 0.016340	0.016179	0.016585	0.016076	0.016086

V2 - test run on 25 targets

V1 - test 

-----------------------

MoA48_1

V8 L1 logreg C=2, blends = 1

V7 L1 logreg C=0.5, blends = 1, 0.016723 0.016541

V6 L1 logreg C=1, blends = 1, Loss1 0.016308 Loss2 0.016247

V5 blends 1, C = [0.25, 0.5,1,2, 4, 8, 16, 32 ] Best No blend: 0.01618 Best Blend 0.01612 Blend gain best to best in 5-th digits: 6.0 Best blend 0.01612 at C-multiplier 4 Best blend at N 4 0.016197

v4 20 blends, C=1, No blend: 0.01619 Blend 0.01611 Blend gain to mean in 5-th digits: 8.0

V3 - same as V2 - difference due to randomness - 0.016194 0.016169 - randomness affect less than 5th digit , it is strange - an a plot from v4 we see about 3 5th digits variation.

V2 - 1,1,1 Loss1 0.016192 Loss2 (correct worse than constant) 0.016162 - difference - 3 * 5th digits quite small

V1 - test run 25 targets

MoA46_3

V4 - filter out worse constant CV 0.016175 LB 0.01951

V3 - Fit on full train, C=1 n_blends =1, LB 0.01950 CV 0.017603 is approximate

From MoA46

V4 n_blends = 1, C_blends = [2] (no blend) LB 0.01959 CV 0.017740

V3 n_blends = 1, C_blends = [0.5] (no blend) LB 0.01952 CV 0.017705

V2 n_blends = 1, C_blends = [1] (no blend) LB 0.01948, CV 0.017627

V1 - test version does not support blend over folds yet.

In [ ]:
flag_train_on_whole_train_for_the_first_model_in_blend = 0 # train on whole train (without subfolds splits)
    # if 1 that is done for the first ONLY model in loop over n_blends 
    # thus if several C in list that will be done for each C 

n_blends = 1
list4blend_change_optimal_C_by = [0.5, 1, 2, 4 ]# [0.25, 0.5,1,2, 4, 8, 16, 32 ] 


#list_models = ['l2', 'l1']
list_models = ['Const Predict','l1','l2']



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load data with slight preprocessing

In [ ]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import datetime
import time

import matplotlib.pyplot as plt
import seaborn as sns

import time
df = pd.read_csv('/kaggle/input/lish-moa/train_features.csv',index_col = 0)  
df0 = df.copy()
df['cp_type'] = df['cp_type'].map({'trt_cp':1.0, 'ctl_vehicle':1.0}) # Forget about control group  
df['cp_dose'] = df['cp_dose'].map({'D1':0.0, 'D2':1.0})
df['cp_time'] = df['cp_time'].map({24:0.0, 48: .5 , 72:1.0})
X = df.copy()
X_save = X.copy()
df_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv',index_col = 0)
df0_test = df_test.copy()
df_test['cp_type'] = df_test['cp_type'].map({'trt_cp':1.0, 'ctl_vehicle':0.0})
df_test['cp_dose'] = df_test['cp_dose'].map({'D1':0.0, 'D2':1.0})
df_test['cp_time'] = df_test['cp_time'].map({24:0.0, 48: .5 , 72:1.0})

y = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv',index_col = 0 )
y_save = y.copy()
print(y.iloc[:3,:2])
df

In [ ]:
y_save.sum(axis = 0 ).sort_values(ascending = False)

# Function to return list with mask for Chris folds

In [ ]:
# LOAD LIBRARIES (from PIP or Kaggle Dataset)
#! pip install iterative-stratification 
!pip install /kaggle/input/iterative-stratification/iterative-stratification-master/
from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# LOAD FILES
scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')
targets = scored.columns[1:]
scored = scored.merge(drug, on='sig_id', how='left') 

def get_list_folds_masks(SEED = 42, FOLDS = 5):

    # LOCATE DRUGS
    vc = scored.drug_id.value_counts()
    vc1 = vc.loc[vc<=18].index.sort_values()
    vc2 = vc.loc[vc>18].index.sort_values()

    # STRATIFY DRUGS 18X OR LESS
    dct1 = {}; dct2 = {}
    skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, 
              random_state=SEED)
    tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.index[idxV].values}
        dct1.update(dd)

    # STRATIFY DRUGS MORE THAN 18X
    skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, 
              random_state=SEED)
    tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.sig_id[idxV].values}
        dct2.update(dd)

    # ASSIGN FOLDS
    scored['fold'] = scored.drug_id.map(dct1)
    scored.loc[scored.fold.isna(),'fold'] =\
        scored.loc[scored.fold.isna(),'sig_id'].map(dct2)
    scored.fold = scored.fold.astype('int8')

    df_folds = scored[['sig_id','fold']]
    df_folds = df_folds.set_index('sig_id')
    if 0:
        print(df_folds['fold'].value_counts())
    list_folds_masks = []
    for i in range(FOLDS):
        m = df_folds['fold'] == i
        list_folds_masks.append((~m,m))
    if 0:
        for mask_train,mask_test in list_folds_masks:
            print(mask_train.sum(), mask_test.sum(),  mask_train.sum()+ mask_test.sum())

    return list_folds_masks

list_folds_masks = get_list_folds_masks(SEED = 42, FOLDS = 5)
for mask_train,mask_test in list_folds_masks:
    print(mask_train.sum(), mask_test.sum(),  mask_train.sum()+ mask_test.sum())


In [ ]:
import numpy as np
import pandas as pd
# list_C is min from cv3 rs0,1,100 finer grid than before - used in MoA23 v6 and above
list_C_MoA36 = [0.03, 0.003, 0.0002, 0.001, 0.0007, 0.001, 0.001, 0.002, 0.1, 0.005, 0.001, 0.01, 0.5, 0.01, 0.0007, 0.002, 0.001, 0.001, 0.001, 0.001, 0.002, 0.002, 0.007, 0.005, 0.005, 0.01, 0.005, 0.005, 0.003, 0.002, 0.01, 0.005, 0.002, 0.03, 0.0, 0.1, 0.005, 0.05, 0.02, 0.007, 0.003, 0.007, 0.005, 0.001, 0.003, 0.005, 0.02, 0.01, 0.01, 0.003, 0.001, 0.01, 0.02, 0.02, 0.001, 0.003, 0.005, 0.001, 0.007, 0.01, 0.0003, 0.0002, 0.005, 0.007, 0.002, 0.2, 0.005, 0.001, 0.002, 0.005, 0.005, 0.001, 0.002, 0.007, 0.007, 0.007, 0.005, 0.002, 0.003, 0.002, 0.02, 0.003, 0.0, 0.002, 0.005, 0.007, 0.05, 0.005, 0.02, 0.01, 0.05, 0.003, 0.007, 0.003, 0.002, 0.02, 0.005, 0.02, 0.002, 0.0007, 0.0007, 0.01, 0.002, 0.01, 0.002, 0.001, 0.1, 0.03, 0.005, 0.05, 0.02, 0.02, 0.01, 0.001, 0.003, 0.002, 0.01, 0.001, 0.02, 0.02, 0.005, 0.01, 0.003, 0.001, 0.003, 0.03, 0.07, 0.02, 0.002, 0.003, 0.0005, 0.002, 0.02, 0.007, 0.003, 0.005, 0.005, 0.01, 0.003, 0.003, 0.003, 0.01, 0.01, 0.002, 0.002, 0.0003, 0.02, 0.005, 0.01, 0.01, 0.01, 0.003, 0.003, 0.007, 0.007, 0.001, 0.0007, 0.01, 0.003, 0.007, 0.03, 0.003, 0.001, 1.0, 0.002, 0.02, 0.005, 0.0, 0.001, 0.05, 0.007, 0.05, 0.01, 0.03, 0.003, 0.007, 0.0005, 0.002, 0.005, 0.005, 0.002, 0.005, 0.001, 0.007, 0.007, 0.0007, 0.02, 0.005, 0.03, 0.005, 0.007, 0.001, 0.003, 0.005, 0.007, 0.002, 0.001, 0.02, 0.003, 0.01, 0.003, 0.02, 0.01, 0.0005, 0.1, 0.0007]
list_C_MoA45_V9_l2 = [0.036, 0.005183999999999999, 5.787037037037037e-05, 0.0003348979766803842, 0.0005787037037037038, 0.0002790816472336535, 0.00048225308641975317, 0.0003348979766803842, 0.03, 0.005183999999999999, 0.0008333333333333334, 0.0036, np.nan, 0.0036, 0.0006944444444444446, 0.0025, 0.001, 0.0005787037037037038, 0.0012, 0.0006220799999999999, 0.003, 0.0014467592592592596, np.nan, 2.790816472336535e-06, 0.0025, 0.0144, 0.0005787037037037038, 0.00043199999999999993, 0.0025, 0.0017279999999999997, 0.0025, 2.790816472336535e-06, 0.00035999999999999997, np.nan, np.nan, 0.020833333333333336, 0.0020833333333333337, 0.005787037037037038, 0.0144, np.nan, 0.0017279999999999997, 0.0048225308641975315, 0.00432, 0.0012, 0.0020736, 0.0036, 0.008333333333333333, 0.005183999999999999, 0.0017361111111111112, 0.003, 0.00020833333333333335, 0.0025, 0.008333333333333333, np.nan, 0.0005787037037037038, 0.0020833333333333337, 0.00017279999999999997, 0.00048225308641975317, 0.0005183999999999999, 0.00432, 0.00043199999999999993, 8.333333333333334e-05, 0.005183999999999999, 0.003, 0.00035999999999999997, 0.0144, 0.0036, 0.0006220799999999999, 0.00025, np.nan, 0.006220799999999999, 0.001, 0.000144, 0.006944444444444446, 0.0036, np.nan, 0.00048225308641975317, 0.0008333333333333334, 0.0005183999999999999, 0.0017279999999999997, 0.017279999999999997, np.nan, np.nan, 0.0014399999999999999, 0.0025, 0.005787037037037038, 0.036, 0.0036, 0.006944444444444446, 0.005183999999999999, 0.020833333333333336, 0.0006944444444444446, 0.006944444444444446, 8.333333333333334e-05, 0.0008333333333333334, 0.0144, 0.006220799999999999, 0.00025, 0.0017279999999999997, 0.0008333333333333334, 0.0017279999999999997, 0.005787037037037038, 0.000144, 0.005183999999999999, 0.0014399999999999999, 0.00048225308641975317, 0.05787037037037038, 0.0144, 0.005183999999999999, 0.06220799999999999, 0.006220799999999999, 0.005183999999999999, 2.790816472336535e-06, 0.00025, 0.0012056327160493829, 0.0005787037037037038, 0.005183999999999999, 6.944444444444446e-05, 0.01, 0.012, np.nan, np.nan, 0.00017279999999999997, 0.001, 2.790816472336535e-06, np.nan, 0.0144, 0.00432, 0.00012, 0.00043199999999999993, 0.00012, 0.0006944444444444446, 0.020833333333333336, 0.005183999999999999, 0.0017279999999999997, 0.003, 0.0020833333333333337, np.nan, 0.0036, 2.790816472336535e-06, 0.0025, 0.00043199999999999993, 0.00043199999999999993, 0.0003, 2.7908164723365347e-05, 0.00025, 0.012, 4.8225308641975325e-06, 0.005787037037037038, 0.01, 0.0014399999999999999, 0.0025, 0.0005183999999999999, 0.0017361111111111112, 2.790816472336535e-06, 0.0002790816472336535, 0.0005787037037037038, 0.006944444444444446, 0.0005787037037037038, 0.005787037037037038, 0.025, 0.0014399999999999999, 0.00025, 1.0, 5.183999999999999e-05, np.nan, 0.00048225308641975317, 2.790816472336535e-06, 0.0008333333333333334, 0.036, 3.348979766803842e-06, 0.017361111111111112, np.nan, 0.017361111111111112, 0.0017361111111111112, 2.790816472336535e-06, 0.00017279999999999997, 0.0012, 0.00024883199999999994, 0.001, 0.0020833333333333337, 8.333333333333334e-05, 0.0005787037037037038, 0.001, 0.0025, np.nan, 2.790816472336535e-06, 0.0017279999999999997, 0.020833333333333336, 0.0020833333333333337, 0.0036, 0.00035999999999999997, 0.005787037037037038, 0.0025, 0.008333333333333333, 0.003, np.nan, 0.0017361111111111112, 0.0010749542399999996, 0.008333333333333333, 1.4400000000000001e-05, np.nan, 0.006220799999999999, 0.0006944444444444446, 0.05787037037037038, 0.00020833333333333335]
list_C_l2 = list_C_MoA45_V9_l2
list_C_MoA45_V11_l1_seed21113 = [0.432, 0.144, 0.08333333333333334, 0.06220799999999999, 0.036, 0.05183999999999999, 0.05787037037037038, 0.05787037037037038, 0.4822530864197532, 0.1, 0.043199999999999995, 0.1, np.nan, 0.05787037037037038, 0.1736111111111111, 0.25, 0.06944444444444446, 0.06220799999999999, 0.05787037037037038, 0.06944444444444446, 0.08333333333333334, 0.06944444444444446, np.nan, 0.06220799999999999, 0.17279999999999998, 0.20833333333333334, 0.144, 0.08333333333333334, 0.08333333333333334, 0.1, 0.1, 0.036, 0.1, np.nan, np.nan, 0.5183999999999999, 0.05787037037037038, 0.3, 0.144, np.nan, 0.06944444444444446, 0.08333333333333334, 0.144, 0.036, 0.05787037037037038, 0.1, 0.36, 0.12, 0.08333333333333334, 0.06944444444444446, 0.1, 0.144, 0.12, np.nan, 0.03, 0.1, 0.05787037037037038, 0.08333333333333334, 0.06944444444444446, 0.25, 0.17279999999999998, 0.036, 0.17279999999999998, 0.08333333333333334, 0.05787037037037038, 0.432, 0.1, 0.08333333333333334, 0.05183999999999999, np.nan, 0.144, 0.020833333333333336, 0.03, 0.144, 0.12, np.nan, 0.12, 0.025, 0.06220799999999999, 0.043199999999999995, 0.17279999999999998, np.nan, np.nan, 0.06220799999999999, 0.1, 0.12, 0.4822530864197532, 0.1, 0.08333333333333334, 0.06220799999999999, 0.5787037037037037, 0.17279999999999998, 0.12, 0.043199999999999995, 0.05183999999999999, 0.144, 0.1, 0.12, 0.05787037037037038, 0.036, 0.144, 0.12, 0.043199999999999995, 0.08333333333333334, 0.08333333333333334, 0.036, 0.4822530864197532, 0.3, 0.08333333333333334, 0.3, 0.12, 0.12, 0.03, 0.06944444444444446, 0.06944444444444446, 0.05787037037037038, 0.12, 0.05787037037037038, 0.12, 0.08333333333333334, np.nan, np.nan, 0.05183999999999999, 0.17279999999999998, 0.043199999999999995, np.nan, 0.12, 0.08333333333333334, 0.1, 0.12, 0.144, 0.06220799999999999, 0.3, 0.12, 0.1, 0.1, 0.04822530864197532, np.nan, 0.17279999999999998, 0.06944444444444446, 0.12, 0.1736111111111111, 0.12, 0.05787037037037038, 0.043199999999999995, 0.08333333333333334, 0.144, 0.06944444444444446, 0.1, 0.1, 0.144, 0.05183999999999999, 0.1, 0.05183999999999999, 0.05183999999999999, 0.06944444444444446, 0.043199999999999995, 0.06944444444444446, 0.1, 0.1, 0.1736111111111111, 0.1, 0.05183999999999999, 2.9859839999999993, 0.06220799999999999, np.nan, 0.043199999999999995, 0.1, 0.06944444444444446, 0.5183999999999999, 0.12, 0.144, np.nan, 0.25, 0.08333333333333334, 0.08333333333333334, 0.025, 0.043199999999999995, 0.06944444444444446, 0.12, 0.12, 0.1, 0.036, 0.144, 0.05787037037037038, np.nan, 0.06944444444444446, 0.08333333333333334, 0.36, 0.12, 0.12, 0.1, 0.20833333333333334, 0.1, 0.06944444444444446, 0.20833333333333334, np.nan, 0.06944444444444446, 0.08333333333333334, 0.12, 0.06220799999999999, np.nan, 0.06220799999999999, 0.1, 0.4822530864197532, 0.08333333333333334]
list_C_MoA45_V10_l1_seed100 = [0.432, 0.144, 0.08333333333333334, 0.036, 0.05183999999999999, 0.05183999999999999, 0.06944444444444446, 0.04822530864197532, 0.5787037037037037, 0.1, 0.043199999999999995, 0.1, np.nan, 0.08333333333333334, 0.20833333333333334, 0.25, 0.05787037037037038, 0.05183999999999999, 0.06944444444444446, 0.08333333333333334, 0.08333333333333334, 0.06944444444444446, np.nan, 0.06220799999999999, 0.144, 0.20833333333333334, 0.144, 0.08333333333333334, 0.08333333333333334, 0.12, 0.1, 0.036, 0.06944444444444446, np.nan, np.nan, 0.5183999999999999, 0.05787037037037038, 0.25, 0.20736, np.nan, 0.06944444444444446, 0.144, 0.144, 0.05183999999999999, 0.08333333333333334, 0.1, 0.3, 0.1, 0.08333333333333334, 0.06944444444444446, 0.1, 0.08333333333333334, 0.17279999999999998, np.nan, 0.03, 0.08333333333333334, 0.06944444444444446, 0.08333333333333334, 0.06944444444444446, 0.20833333333333334, 0.17279999999999998, 0.043199999999999995, 0.17279999999999998, 0.08333333333333334, 0.08333333333333334, 0.36, 0.1, 0.08333333333333334, 0.05183999999999999, np.nan, 0.17279999999999998, 0.03, 0.036, 0.144, 0.17279999999999998, np.nan, 0.12, 0.025, 0.06220799999999999, 0.05183999999999999, 0.17279999999999998, np.nan, np.nan, 0.07464959999999998, 0.1, 0.12, 0.4822530864197532, 0.1, 0.08333333333333334, 0.05183999999999999, 0.40187757201646096, 0.17279999999999998, 0.12, 0.043199999999999995, 0.06220799999999999, 0.144, 0.1, 0.12, 0.08333333333333334, 0.043199999999999995, 0.144, 0.1, 0.043199999999999995, 0.12, 0.08333333333333334, 0.043199999999999995, 0.40187757201646096, 0.25, 0.08333333333333334, 0.3, 0.1, 0.12, 0.03, 0.06944444444444446, 0.06944444444444446, 0.05787037037037038, 0.1, 0.05787037037037038, 0.1, 0.08333333333333334, np.nan, np.nan, 0.05183999999999999, 0.17279999999999998, 0.043199999999999995, np.nan, 0.12, 0.08333333333333334, 0.08333333333333334, 0.12, 0.144, 0.06220799999999999, 0.3, 0.1, 0.1, 0.12, 0.05787037037037038, np.nan, 0.144, 0.06944444444444446, 0.12, 0.1736111111111111, 0.12, 0.06944444444444446, 0.036, 0.1, 0.17279999999999998, 0.06944444444444446, 0.1, 0.1, 0.144, 0.06220799999999999, 0.1, 0.05183999999999999, 0.05183999999999999, 0.05787037037037038, 0.043199999999999995, 0.08333333333333334, 0.08333333333333334, 0.08333333333333334, 0.20833333333333334, 0.1, 0.05183999999999999, 1.44, 0.05183999999999999, np.nan, 0.036, 0.1, 0.06944444444444446, 0.3, 0.144, 0.144, np.nan, 0.20833333333333334, 0.08333333333333334, 0.06944444444444446, 0.03, 0.036, 0.06944444444444446, 0.08333333333333334, 0.12, 0.1, 0.03, 0.144, 0.05787037037037038, np.nan, 0.06944444444444446, 0.08333333333333334, 0.432, 0.12, 0.12, 0.1, 0.25, 0.1, 0.1, 0.25, np.nan, 0.06944444444444446, 0.144, 0.12, 0.05183999999999999, np.nan, 0.06220799999999999, 0.1, 0.3348979766803842, 0.08333333333333334]

l1 = list_C_MoA45_V11_l1_seed21113
l2 = list_C_MoA45_V10_l1_seed100
list_C_l1 = [ np.min( [l1[i], l2[i]] ) for i in range(len(l1) )  ]

pd.Series(list_C_l1).describe() #list_C

In [ ]:
for c in list_C_l1[:10]:
    print(c, type(c))
    

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Core simulation

Logreg for each targets

with possible blend over different C and different splits on subfolds 


In [ ]:

def modeling(X,y, model_id = 'Const Predict', verbose = 0):
    if model_id == 'Const Predict':
        y_pred_oof = np.ones_like(y)*np.mean(y) 
        mn = y[df0.cp_type=='trt_cp'].mean()
        y_pred_submit = np.ones( len(df_test) ) * mn # y.mean() 

    if (model_id == 'l1') or  (model_id == 'l2') :
        if (model_id == 'l1'):         C = list_C_l1[cnt_target]
        else: C = list_C_l2[cnt_target]
        
        if (np.isnan(C) or (C == 0) ):
            return modeling(X,y, model_id = 'Const Predict')
        
        y_pred_submit = np.zeros( len(df_test) )
        cnt_blend_submit = 0
        y_pred_oof_blend = np.zeros_like(y,dtype = float)
        cnt_blend_oof = 0
        for C_effective in C*np.array(list4blend_change_optimal_C_by): # np.linspace(0.5,1,10): # np.array([0.5,1, 2]):
        
            if (model_id == 'l1'):
                model = LogisticRegression( C = C_effective , penalty='l1', solver = 'liblinear' )  
            else: 
                model = LogisticRegression( C = C_effective )  
            for cnt_blends in range(n_blends):
                rs = np.random.randint(10**7)
                #skf = StratifiedKFold(n_splits=3, shuffle=True, random_state= rs )
                list_folds_masks = get_list_folds_masks(SEED = rs, FOLDS = 5)
                y_pred_oof = np.zeros_like(y,dtype = float)
                list_loss_train = []
                list_loss_test = []
                
                #for train_index, test_index in skf.split(X, y):
                for mask_train,mask_test in list_folds_masks:
                    X_train, X_test = X[mask_train], X[mask_test]
                    y_train, y_test = y[mask_train], y[mask_test]
                    #X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                    #y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
                    model.fit(X_train,y_train)
                    #y_pred_oof[ test_index ] = model.predict_proba(X_test)[:,1]
                    y_pred_oof[ mask_test ] = model.predict_proba(X_test)[:,1]

                    # we have an option to fit the model on the whole train, and predict on submit
                    # we do it if flag_... >0 and for the first blend in loop iteration  
                    # thus for these conditions submit is not calculated here 
                    if not(  (flag_train_on_whole_train_for_the_first_model_in_blend > 0 ) and (  cnt_blends == 0) ):
                        y_pred_submit = (y_pred_submit*cnt_blend_submit + model.predict_proba(df_test)[:,1])/(cnt_blend_submit+1) # blend
                        cnt_blend_submit += 1

                    list_loss_train.append( log_loss(y_train,  model.predict_proba(X_train)[:,1] , labels = [0,1]))
                    list_loss_test.append( log_loss(y_test,  model.predict_proba(X_test)[:,1], labels = [0,1] ) )
                        
                    if verbose >= 100:
                        print( np.round(list_loss_test,6) )
                        
                if flag_train_on_whole_train_for_the_first_model_in_blend > 0:
                    if cnt_blends == 0:
                        model.fit(X,y)
                        y_pred_submit = (y_pred_submit*cnt_blend_submit + model.predict_proba(df_test)[:,1])/(cnt_blend_submit+1)                        
                        cnt_blend_submit += 1

    return y_pred_oof, y_pred_submit

In [ ]:
from scipy.optimize import minimize, fsolve
bnds = ((1, None), (0, None))
def f0(x):
    return x[0]**2+x[1]**2
res_scipy = minimize( f0,  [10,1] , bounds=bnds,)
res_scipy

In [ ]:
def blending_simple( list_preds ):
    
    y_pred_blend = np.zeros_like(y,dtype = float)
    y_pred_submit = np.zeros( len(df_test), dtype = float )
    c = 0
    s0 = log_loss(y, np.ones_like(y)*np.mean(y) )
    for i,t in enumerate(list_preds):
        y_pred_oof = t[0]
        s = log_loss(y, y_pred_oof )
        if s >= s0: continue
        c+=1
        #df_stat.loc[cnt4df_stat,'Loss OOF'+str(i)] = s # should do it in stat function
        y_pred_blend +=  y_pred_oof
        y_pred_submit += t[1]
        
    cnt4df_stat = df_stat.index[-1]# len(df_stat)
    if c > 0:
        y_pred_blend /= c
        y_pred_submit /= c
        df_stat.loc[cnt4df_stat,'Loss OOF Blend Simple 2'] = log_loss(y, y_pred_blend )
        df_stat.loc[cnt4df_stat,'Blend Simple 2 Used Const'] = 0
    else:
        df_stat.loc[cnt4df_stat,'Loss OOF Blend Simple 2'] = s0
        y_pred_blend = np.ones_like(y)*np.mean(y) 
        mn = y[df0.cp_type=='trt_cp'].mean()
        y_pred_submit = np.ones( len(df_test) ) * mn # y.mean() 
        df_stat.loc[cnt4df_stat,'Blend Simple 2 Used Const'] = 1
        
        
        
    return y_pred_blend, y_pred_submit 
    

In [ ]:

def blending( list_preds) :
    
    if 0:
        y_pred_blend = np.ones_like(y)*np.mean(y) 
        mn = y[df0.cp_type=='trt_cp'].mean()
        y_pred_submit = np.ones( len(df_test) ) * mn # y.mean() 

    
    rs = np.random.randint(10**7)
    #skf = StratifiedKFold(n_splits=3, shuffle=True, random_state= rs )
    list_folds_masks = get_list_folds_masks(SEED = rs, FOLDS = 5)
    mask_train, mask_test = list_folds_masks[0][:2]
    
    list_scores = [np.nan, np.nan]
    def func2optm(x):
        y_pred_blend = np.zeros_like(y,dtype = float)
        for i,y_pred in enumerate(list_preds):
            y_pred_blend += x[i]*list_preds[i][0]
        score_train = log_loss(y[mask_train], y_pred_blend[mask_train], labels = [0,1])
        score_test = log_loss(y[mask_test], y_pred_blend[mask_test] , labels = [0,1] )
        list_scores[0] = score_train
        list_scores[1] = score_test
        return score_train
    
    x0 = np.ones(len(list_preds)) * (1.0/len(list_preds)) 
    bnds = []
    for i,y_pred in enumerate(list_preds):
        bnds.append((0,None))
    res_scipy = minimize(fun = func2optm, x0=  x0, bounds = bnds  ) 
    
    
    y_pred_blend = np.zeros_like(y,dtype = float)
    for i,y_pred in enumerate(list_preds):
        y_pred_blend += res_scipy.x[i]*list_preds[i][0]
    y_pred_submit = np.zeros( len(df_test) ) 
    for i,y_pred in enumerate(list_preds):
        y_pred_submit += res_scipy.x[i]*list_preds[i][1]
    if log_loss(y, y_pred_blend) >= log_loss(y, np.ones_like(y)*np.mean(y)  ):
        y_pred_blend = np.ones_like(y)*np.mean(y) 
        mn = y[df0.cp_type=='trt_cp'].mean()
        y_pred_submit = np.ones( len(df_test) ) * mn # y.mean() 

    cnt4df_stat = df_stat.index[-1]# len(df_stat)
    df_stat.loc[cnt4df_stat,'Blend Weighted'] = log_loss( y, y_pred_blend )
    for i,y_pred in enumerate(range(len(res_scipy.x))):
        df_stat.loc[cnt4df_stat,'W'+str(i)] = res_scipy.x[i]
        
    return y_pred_blend, y_pred_submit 


In [ ]:
def df_stat_update_0( ):
    cnt4df_stat = len(df_stat)+1

    df_stat.loc[cnt4df_stat,'Target'] = target_name
    df_stat.loc[cnt4df_stat,'Target Sum'] = np.sum(y) # target_name
    df_stat['Target Sum'] = df_stat['Target Sum'].astype(int) # np.sum(y) # target_name
    df_stat.loc[cnt4df_stat,'Loss Const Predict'] = log_loss( y, np.ones_like(y)*np.mean(y) ) 
    df_stat.loc[cnt4df_stat,'Seconds Passed'] = np.round(time.time()-t00,1)

def    df_stat_update_1( y_pred, model_inf = 'Blend' ):
    cnt4df_stat = df_stat.index[-1]# len(df_stat)
    df_stat.loc[cnt4df_stat,'Loss '+model_inf] = log_loss(y, y_pred )

def df_stat_update_list_preds( list_preds ):
    cnt4df_stat = df_stat.index[-1]# len(df_stat)

    y_blend_simple = np.zeros_like(y,dtype = float)
    for i,t in enumerate(list_preds):
        y_pred_oof = t[0]
        df_stat.loc[cnt4df_stat,'Loss OOF'+str(i)] = log_loss(y, y_pred_oof )
        y_blend_simple +=  y_pred_oof
    y_blend_simple /= len(list_preds)
    df_stat.loc[cnt4df_stat,'Loss OOF Blend Simple'] = log_loss(y, y_blend_simple )
    
    y_blend_simple = np.zeros_like(y,dtype = float)
    c = 0
    s0 = log_loss(y, np.ones_like(y)*np.mean(y) )
    for i,t in enumerate(list_preds):
        y_pred_oof = t[0]
        s = log_loss(y, y_pred_oof )
        if s >= s0:
            df_stat.loc[cnt4df_stat,'Loss OOF Corrected Compare to Const'+str(i)] = s0
            continue
        c+=1
        df_stat.loc[cnt4df_stat,'Loss OOF Corrected Compare to Const'+str(i)] = s
        y_blend_simple +=  y_pred_oof
    if c > 0:
        y_blend_simple /= c
        df_stat.loc[cnt4df_stat,'Loss OOF Blend Simple Better Cnst'] = log_loss(y, y_blend_simple )
        df_stat.loc[cnt4df_stat,'Blend Simple Used Const'] = 0
    else:
        df_stat.loc[cnt4df_stat,'Loss OOF Blend Simple Better Cnst'] = s0
        df_stat.loc[cnt4df_stat,'Blend Simple Used Const'] = 1
        


In [ ]:
#y1 = y_pred_oof.copy()


In [ ]:
if 0:
    l = []
    s = log_loss(y,y1)
    print(s, y.mean())
    y_in = y1.copy()

    s1 = np.inf
    m_best = -1
    a_best = None
    for a in np.linspace(-0.01,1,100):
        print(a,s1, a_best, m_best)
        for m in np.linspace(0.5,2,20):
            y_in = y1.copy()*m+a

            for a_min in np.linspace(0,1,200):
                y2 = np.clip(y_in, a_min,1)
                s = log_loss(y,y2)
                l.append(s)
                if s <= np.min(l):
                    a_best = a_min
                    s_best = s
                    y_best = y2.copy()
            #print(a_best,s_best, 'a_min s_best')        
            #print()

            l = []
            y_in = y_best.copy()
            for a_max in np.linspace(0,1,200):
                y2 = np.clip(y_in, 0,  a_max)# ,1)
                s = log_loss(y,y2)
                l.append(s)
                if s <= np.min(l):
                    a_best = a_max
                    s_best = s
                    y_best = y2
            #print(a_best,s_best) 
            if s_best < s1:
                print(s1,'s1 before', s_best,'s_best')
                s1 = s_best
                #s1 =np.min([s_best, s1])
                m_best = m
                a_best = a
                print(a, m_best, s_best ,'upd')
    s1 , m_best
    
    
    

In [ ]:
# Core run
from sklearn.linear_model import LogisticRegression
verbose = 0#  100
# flag_train_on_whole_train_for_the_first_model_in_blend = 1

df_submit = pd.DataFrame(index = df_test.index)
df_train_oof_pred = pd.DataFrame(index = df0.index) 

t00 = time.time(); t0 = time.time()
df_stat = pd.DataFrame()
cnt4df_stat = 0
df_stat2 = pd.DataFrame()
cnt4df_stat2 = 0
for cnt_target, target_name in enumerate(y_save.columns):#enumerate(['dopamine_receptor_antagonist']) : # y_save.columns):
    # if target_name != 'dopamine_receptor_antagonist': continue
    y = y_save[target_name]
    df_stat_update_0( )
    #    C = 0.003

    list_models =  ['Const Predict','l1','l2']
    list_preds = []
    for model_id in list_models:
        #print( model_id )
        y_pred_oof, y_pred_submit = modeling( X, y, model_id , verbose = verbose)
        list_preds.append( (y_pred_oof, y_pred_submit) )
    df_stat_update_list_preds( list_preds )
    
    #y_pred_oof, y_pred_submit = blending( list_preds) 
    y_pred_oof, y_pred_submit = blending_simple( list_preds )    
    df_stat_update_1( y_pred_oof, 'Blend' )
 
    df_train_oof_pred.loc[:,target_name] = y_pred_oof
    df_submit.loc[:,target_name] = y_pred_submit        
    
    str_inf = ''#  get_str_info( )
    print(cnt_target,   target_name, str_inf, np.round(time.time()-t00,1), 'seconds' )
    
total_time = time.time()-t00    
#df_stat.to_csv("df_stat.csv")
print(np.round(total_time,0), np.round(total_time/60,0), np.round(total_time/3600,0),'seconds,minutes, hours total passed') ; 
df_train_oof_pred.to_csv('df_train_oof_pred.csv')
df_stat2.to_csv('df_stat2.csv')
df_stat.to_csv('df_stat.csv')

df_submit

#df_stat2.describe()
df_stat

In [ ]:
print(np.round(total_time,0), np.round(total_time/60,0), np.round(total_time/3600,0),'seconds,minutes, hours total passed') ; 


In [ ]:
df_stat[['Blend Simple Used Const', 'Blend Simple 2 Used Const',]].sum()

In [ ]:
df_stat.describe()

In [ ]:
df_stat

In [ ]:
print(np.round(total_time,0), np.round(total_time/60,0), np.round(total_time/3600,0),'seconds,minutes, hours total passed') ; 
df_stat.to_csv('df_stat.csv')

In [ ]:
for f in y_save.columns:
    #df_submit[f] = train_targets.loc[ train_features.cp_type=='trt_cp', f].mean()
    df_submit.loc[ df0_test.cp_type!='trt_cp' ,f] = 0
df_submit#.head()

In [ ]:
df_submit.to_csv("submission.csv")
